In [6]:
import spacy
import random
from spacy.training.example import Example
from spacy.util import minibatch, compounding
import json
with open('data_full.json', 'r', encoding='utf-8') as f:
    TRAIN_DATA = json.load(f)
nlp = spacy.blank("id")
ner = nlp.add_pipe("ner")
for item in TRAIN_DATA:
    for start, end, label in item['entities']:
        ner.add_label(label)
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*unaffected_pipes):
    optimizer = nlp.begin_training()
    for itn in range(150): 
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            examples = []
            for d in batch:
                doc = nlp.make_doc(d['text'])
                example = Example.from_dict(doc, {"entities": d['entities']})
                examples.append(example)
            nlp.update(examples, drop=0.5, losses=losses)
        print(f"Losses at iteration {itn}: {losses}")

Losses at iteration 0: {'ner': np.float32(14065.617)}
Losses at iteration 1: {'ner': np.float32(10343.373)}
Losses at iteration 2: {'ner': np.float32(8645.48)}
Losses at iteration 3: {'ner': np.float32(7764.442)}
Losses at iteration 4: {'ner': np.float32(7050.838)}
Losses at iteration 5: {'ner': np.float32(6634.4854)}
Losses at iteration 6: {'ner': np.float32(6186.873)}
Losses at iteration 7: {'ner': np.float32(5917.1665)}
Losses at iteration 8: {'ner': np.float32(5724.998)}
Losses at iteration 9: {'ner': np.float32(5478.1646)}
Losses at iteration 10: {'ner': np.float32(5293.305)}
Losses at iteration 11: {'ner': np.float32(5147.912)}
Losses at iteration 12: {'ner': np.float32(4913.897)}
Losses at iteration 13: {'ner': np.float32(4717.1787)}
Losses at iteration 14: {'ner': np.float32(4637.5503)}
Losses at iteration 15: {'ner': np.float32(4415.6865)}
Losses at iteration 16: {'ner': np.float32(4521.2607)}
Losses at iteration 17: {'ner': np.float32(4378.7397)}
Losses at iteration 18: {'ner

In [7]:
nlp.to_disk("model")
print("Model saved to 'model'")

Model saved to 'model'


In [8]:
text = """
Keamanan dan perlindungan data menjadi isu hangat usai Pusat Data Nasional (PDN) diretas beberapa waktu lalu. Sehingga, semua pihak termasuk perusahaan Badan Usaha Milik Negara (BUMN) perlu mewaspadai serangan ransomware.
Staf Khusus Menteri BUMN Arya Sinulingga mengatakan, Menteri BUMN Erick Thohir meminta semua perusahaan pelat merah memperkuat sistem keamanannya.

"Kita dengan kejadian PDN Ini memang Pak Menteri juga meminta kita semua memperkuat keamanan sistemnya semua," ujarnya di Jakarta, Kamis (4/7/2024).
Perusahaan negara di semua sektor mulai dari sektor perbankan, PT Pertamina (Persero), Telkom, dan yang lainnya diminta untuk tidak meremehkan keamanan sistem data.

"Apa pun ceritanya keamanan siber ini penting dan ini kejadian walaupun kemarin kita tidak kena di BUMN tapi harus jadi peringatan bagi semua untuk memperkuat," sebutnya.

Menurutnya, backup data perlu dilakukan untuk mengantisipasi hal yang tidak diinginkan di kemudian hari. "Backup mandatori lah itu yang harus dilakukan BUMN supaya kalau ada apa apa bisa menggantikan dengan cepat," pungkasnya.

"""

doc = nlp(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])


Entities [('Data Nasional (PDN)', 'ORGANIZATION'), ('Badan Usaha Milik Negara', 'ORGANIZATION'), ('BUMN', 'ORGANIZATION'), ('Arya Sinulingga', 'PERSON'), ('BUMN', 'ORGANIZATION'), ('Erick Thohir', 'PERSON'), ('Jakarta', 'LOCATION'), ('PT Pertamina (Persero)', 'ORGANIZATION'), ('Telkom', 'ORGANIZATION'), ('BUMN', 'ORGANIZATION'), ('BUMN', 'ORGANIZATION')]
